# Tesis Demo

Se importan las librerias respectivas a **pyspark**.

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext('local[*]')
sqlContext = SQLContext(sc)

El siguiente bloque de codigo importa el modulo que integra las funciones desarrolladas:

In [ ]:
module_path = "/home/jovyan/work/herramientas/tesis_lib.py"
sc.addPyFile(module_path)
import tesis_lib as tools

El siguiente bloque ejecuta una función del modulo que se acaba de cargar, verificando su correcto funcionamiento el resultado debe ser **0.9**.

In [ ]:
tools.lexical_diversity("Hola Mundo")

En el siguiente bloque debe indicar las carpetas respectivas a tweets **SPAM** y **NO SPAM** para el entrenamiento del clasificador de tweets:

In [ ]:
spamFile = "/home/jovyan/work/herramientas/datos/spam/spam"
noSpamFile = "/home/jovyan/work/herramientas/datos/spam/no_spam"
juez_spam = tools.entrenar_spam(sc, sqlContext, spamFile, noSpamFile)

En el siguiente bloque se entrenará el clasificador de timelines, para lo cual se deben especificar las direcciones respectivas a las carpetas que contienen los timelines de **humanos, ciborgs y bots**:

In [ ]:
directorios = {'humanos': '/home/jovyan/work/herramientas/datos/categorizados/humano',
               'ciborgs': '/home/jovyan/work/herramientas/datos/categorizados/ciborg',
               'bots': '/home/jovyan/work/herramientas/datos/categorizados/bot'}
juez_usuarios = tools.entrenar_juez(sc, sqlContext, juez_spam, directorios)

Obteniendo las caracteristicas de los usuarios:

In [ ]:
timelinesInput = '/home/jovyan/work/herramientas/datos/timelines'

features = tools.timeline_features(sc, juez_spam, timelinesInput).cache()

In [ ]:
a = features.first()

En el siguiente bloque de codigo se realizará la evaluación de un usuario para categorizarlo:
* Humano = 0
* Bot = 1
* Ciborg = 2

In [ ]:
resultado, features = tools.evaluar(sc, juez_spam, juez_usuarios, timelinesInput)

resultado.collect()

Probando funcionamiento de **Plotly**

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout, Box, Marker, XAxis, YAxis
import numpy as np
import pandas as pd

init_notebook_mode(connected=True)

In [ ]:
df = pd.read_csv('https://plot.ly/~etpinard/191.csv')

iplot({
    'data': [
        Scatter(x=df[continent+'_Life Expentancy [in years]'],
                y=df[continent+'_Gross Domestic Product per Capita [in USD of the year 2000]'],
                text=df[continent+'_text'],
                marker=Marker(size=df[continent+'_marker.size'], sizemode='area', sizeref=131868,),
                mode='markers',
                name=continent) for continent in ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
    ],
    'layout': Layout(xaxis=XAxis(title='Life Expectancy'), yaxis=YAxis(title='GDP per Capita', type='log'))
}, show_link=False)

In [ ]:
iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])